In [1]:
from bs4 import BeautifulSoup as Soup
import requests
from pandas import DataFrame

response = requests.get('https://www.myfootballfacts.com/premier-league/all-time-premier-league/premier_league_goal_statistics/')

print(response.text)

soup = Soup(response.text)

<!DOCTYPE html>
<html lang="en-US" class="no-js">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="profile" href="https://gmpg.org/xfn/11">
<link rel="pingback" href="https://www.myfootballfacts.com/xmlrpc.php">
<!--[if lt IE 9]>
    <script src="https://html5shiv.googlecode.com/svn/trunk/html5.js"></script>
    <![endif]-->
<title> Premier League Goal Record &amp; Statistics - My Football Facts</title>
<style type="text/css">
					.heateor_sss_button_instagram span.heateor_sss_svg,a.heateor_sss_instagram span.heateor_sss_svg{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
											.heateor_sss_horizontal_sharing .heateor_sss_svg,.heateor_sss_standard_follow_icons_container .heateor_sss_svg{
							color: #fff;
						border-width: 0px;
			border-style: solid;
			border-color: transparent;
		}
					.heateor_sss_horizontal_sharing .heateorSssTCBackground{
				color:#666

In [6]:
# soup is a nested tag, so call find_all on it

tables = soup.find_all('table')

# find_all returns a list of tables
len(tables)

# we want the first one
goals_table = tables[0]

# adp_table another nested tag, so call find_all again
rows = goals_table.find_all('tr')

# header row
# print(rows[2])

# data rows
first_data_row = rows[3]
# print(first_data_row)

rows = goals_table.find_all('tr')

# get columns from first_data_row
# print(first_data_row.find_all('td'))

# comprehension to get raw data out -- each x is simple tag
[str(x.string) for x in first_data_row.find_all('td')]

# put it in a function
def parse_row(row):
    """
    Take in a tr tag and get the data out of it in the form of a list of
    strings.
    """
    return [str(x.string) for x in row.find_all('td')]

# call function
list_of_parsed_rows = [parse_row(row) for row in rows[3:]]
print(list_of_parsed_rows)


[['1992-93', '462', '1,222', '2.65', 'Blackburn Rovers', '68', 'Manchester United', '31', 'Middlesbrough', '75', 'Arsenal', '40'], ['1993-94', '462', '1,195', '2.59', 'Manchester United', '80', 'Arsenal', '28', 'Swindon Town', '100', 'Ipswich Town', '35'], ['1994-95', '462', '1,195', '2.59', 'Blackburn Rovers', '80', 'Manchester United', '28', 'Ipswich Town', '93', 'Crystal Palace', '34'], ['1995-96', '380', '988', '2.60', 'Manchester United', '73', 'Arsenal', '32', 'Bolton Wanderers', '71', 'Manchester City', '33'], ['1996-97', '380', '970', '2.55', 'Manchester United', '76', 'Arsenal', '32', 'Middlesbrough', '60', 'Leeds United', '28'], ['1997-98', '380', '1,019', '2.68', 'Manchester United', '73', 'Manchester United', '26', 'Barnsley', '82', 'Wimbledon', '34'], ['1998-99', '380', '959', '2.52', 'Manchester United', '80', 'Arsenal', '17', 'Nottingham Forest', '69', 'Nottingham Forest', '35'], ['1999-2000', '380', '1,060', '2.79', 'Manchester United', '97', 'Liverpool', '30', 'Watford

In [16]:
import pandas as pd

# Convert to dataframe
df = pd.DataFrame(list_of_parsed_rows)
# Remove the last two rows for formatting purposes
df = df.iloc[:-2]

# Clean up formatting
df.columns = ['season', 'matches', 'goals', 'gpm', 'max_score_team',
              'max_score_n', 'min_allow_team', 'min_allow_n',
              'max_allow_team', 'max_allow_n', 'min_score_team',
              'min_score_n']

# Specify the column types
float_cols = ['gpm']
int_cols = ['matches', 'goals', 'max_score_n', 'min_allow_n', 'max_allow_n', 'min_score_n']

# Remove commas 
for col in int_cols:
    df[col] = df[col].str.replace(',', '')

# Identify and replace problematic entries with NaN
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# OPTIONAL: Fill NaNs with a specific value (like 0)
df[int_cols] = df[int_cols].fillna(0).astype(int)

# Convert the float columns
df[float_cols] = df[float_cols].astype(float)

# Display the first few rows of the cleaned dataframe
df.head()

,season,matches,goals,gpm,max_score_team,max_score_n,min_allow_team,min_allow_n,max_allow_team,max_allow_n,min_score_team,min_score_n
0,1992-93,462,1222,2.65,Blackburn Rovers,68,Manchester United,31,Middlesbrough,75,Arsenal,40
1,1993-94,462,1195,2.59,Manchester United,80,Arsenal,28,Swindon Town,100,Ipswich Town,35
2,1994-95,462,1195,2.59,Blackburn Rovers,80,Manchester United,28,Ipswich Town,93,Crystal Palace,34
3,1995-96,380,988,2.60,Manchester United,73,Arsenal,32,Bolton Wanderers,71,Manchester City,33
4,1996-97,380,970,2.55,Manchester United,76,Arsenal,32,Middlesbrough,60,Leeds United,28
